In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from math import sqrt
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
images_dir = '/content/drive/MyDrive/CS229_Project'

In [ ]:
# transform time series dataset into supervised learning
def series_transform(df, lags, n_predict=1):
	data = list()
	for i in range(lags, 0, -1):
		data.append(df.shift(i))
	for i in range(0, n_predict):
		data.append(df.shift(-i))
	all_data = pd.concat(data, axis=1)
	all_data.dropna(inplace=True)
	return all_data.values
 
# split data into train/test
def train_test_split(series):
  train_test_split=.8
  num_train = round(train_test_split * series.shape[0])
  train, test = series[:num_train], series[num_train:]
  return train, test

# fit random forest model and make predictions
def rf_predict(train, test_X):
	train_X, train_y = train[:, :-1], train[:, -1]
	RF = RandomForestRegressor(n_estimators=100, criterion='squared_error', max_features= .5) # hyperparameter tuning: # of trees, error criterion, max features to consider
	RF.fit(train_X, train_y)
	return RF.predict([test_X])[0]
 
# walk-forward validation
def walk_forward_valid(train, test):
	past = list(train)
	forecast = list()
	for i in range(len(test)):
		print('i:',i)
		test_X, test_y = test[i, :-1], test[i, -1]
		price_predict = rf_predict(np.asarray(past), test_X)
		forecast.append(price_predict)
		past.append(test[i])
	return test[:, -1], forecast

def RMSE(actual, predicted):
  # calculate RMSE
  rmse = sqrt(mean_squared_error(actual, predicted))
  print('Test RMSE: %.3f' % rmse)
  return rmse
 
# load the dataset
# series = pd.read_csv('/content/drive/MyDrive/CS229_Project/pacifica_oct_lmp.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
series = pd.read_csv('/content/drive/MyDrive/CS229_Project/bellehaven_2019.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
df = pd.DataFrame(series.values)
data = series_transform(df, lags=25)
train, test = train_test_split(data)
y_test, y_predict = walk_forward_valid(train, test)
rmse = RMSE(y_test, y_predict)

In [ ]:
# plot
plt.plot(y_test, label = 'Real Price')
plt.plot(y_predict, color='red', label = 'Predicted Price')
plt.title('Random Forest LMP Price Prediction')
plt.xlabel('Time [days]')
plt.xticks(ticks=np.arange(0, len(y_predict), 96), labels=[4*int(x) for x in np.arange(0, len(y_predict), 24)/24])

plt.ylabel('Price [$]')
plt.legend()
plt.savefig(f"{images_dir}/RF_25lags_abs_error.png", dpi=500)
plt.show()